# Pill 17 WIKI Side Quest: Advanced architectures in neural networks

#### by Toni Miranda

## Combining CNN and RNN

CNN and RNN have been described by some of the classmates in this subquest. I would like to add on what has been said by briefly exploring when is a good idea to combine this 2 types of deep-learning models, CNN and RNN. This is on the basis of a book chapter: "Deep Learning for Python" by François Chollet, 2017 (Chapter 6. Deep Learning for text and sequences).

According to Chollet (2017) two fundamental deep-learning algorithms for sequence processing are RNN and 1D CNN, the one-dimensional version of the 2D CNN explained already by classmates and typically used for computer vision. Applications of RNN and 1D CNN  algorithm combination include:
- Document classification and timeseries classification (i.e. identifying the topic of an article or the author of a book).
- Timeseries comparisons, such as estimating how closely related two documents are.
- Sequence-to-sequence learning, such as decoding an English sentence into French
- Sentiment analysis, such as classifying the sentiment of tweets or movie reviews as positive or negative (remember the Kaggle competition in this ML course?)
- Timeseries forecasting, such as predicting the future weather at a certain location, given recent weather data

Note on 1D CNN for sequence data:

The convolution layers introduced previously were 2D convolutions, extracting 2D patches from image tensors and applying an identical transformation to every patch. In the same way, 1D convolutions can be used, extracting local 1D patches (subsequences) from sequences as shown in the figure below (*Fig 1*).

![title](./How_1D_Conv_works.png)  
*Fig 1.* How 1D convolution (1D CNN) works: each output timestep is obtained from a temporal patch in the input sequence.

### Combining CNNs and RNNs

#### Sequence processing with 1-Dimension CNN (1D CNN)

Because 1D CNN process input patches independently, they aren’t sensitive to the order of the timesteps (beyond a local scale, the size of the convolution windows), unlike RNNs. To recognize longer-term patterns, it is possible to stack many convolution layers and pooling layers, but according to Chollet, that’s still a fairly weak way to induce order sensitivity because the CNN looks for patterns anywhere in the input timeseries and has no knowledge of the temporal position of a pattern it sees.

One strategy to combine the speed and lightness of CNNs with the order-sensitivity of RNNs is to use a 1D convnet as a preprocessing step before an RNN. See figure below (*Fig 2*).

![title](./Combining_1D_CNN_and_RNN.png)  
*Fig 2.* Combining a 1D CNN and an RNN for processing long sequences.

#### Combining CNNs and RNNs to process long sequences

This is especially beneficial when dealing with sequences that are so long they can’t realistically be processed with RNNs, such as sequences with thousands of steps. The CNN will turn the long input sequence into
much shorter (downsampled) sequences of higher-level features. This sequence of extracted features then becomes the input to the RNN part of the network.

Key take aways on sequence processing with CNN and combining RNN and CNN:
- In the same way that 2D CNN perform well for processing visual patterns in 2D space, 1D CNN perform well for processing temporal patterns. They offer a faster alternative to RNNs on some problems, in particular natural-language processing tasks.
- Typically, 1D CNN are structured much like their 2D equivalents from the world of computer vision.
- Because RNNs are extremely expensive for processing very long sequences, but 1D CNN are cheap, it can be a good idea to use a 1D CNN as a preprocessing step before an RNN, shortening the sequence and extracting useful representations for the RNN to process.

Exemple of implementation
*Note:* It requires Keras and Tensolflow installed. https://keras.io/#installation

Data set can be download from here: https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip

In [4]:
#Inspecting the data of the Jena weather dataset
import os
data_dir = './'
fname = os.path.join(data_dir, 'jena_climate_2009_2016.csv')
f = open(fname)
data = f.read()
f.close()
lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]
print(header)
print(len(lines))

#Parsing data
import numpy as np
float_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(',')[1:]]
    float_data[i, :] = values

#Normalizing data
mean = float_data[:200000].mean(axis=0)
float_data -= mean
std = float_data[:200000].std(axis=0)
float_data /= std

['"Date Time"', '"p (mbar)"', '"T (degC)"', '"Tpot (K)"', '"Tdew (degC)"', '"rh (%)"', '"VPmax (mbar)"', '"VPact (mbar)"', '"VPdef (mbar)"', '"sh (g/kg)"', '"H2OC (mmol/mol)"', '"rho (g/m**3)"', '"wv (m/s)"', '"max. wv (m/s)"', '"wd (deg)"']
420551


In [5]:
#Generator yielding timeseries samples and their targets
def generator(data, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=128, step=6):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(
                min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)
        samples = np.zeros((len(rows),
                           lookback // step,
                           data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
        yield samples, targets

In [23]:
#Preparing the training, validation, and test generators
lookback = 1440
step = 6
delay = 144
batch_size = 128

train_gen = generator(float_data,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=200000,
                      shuffle=True,
                      step=step,
                      batch_size=batch_size)
val_gen = generator(float_data,
                    lookback=lookback,
                    delay=delay,
                    min_index=200001,
                    max_index=300000,
                    step=step,
                    batch_size=batch_size)
test_gen = generator(float_data,
                     lookback=lookback,
                     delay=delay,
                     min_index=300001,
                     max_index=None,
                     step=step,
                     batch_size=batch_size)

val_steps = (300000 - 200001 - lookback)
test_steps = (len(float_data) - 300001 - lookback)



In [ ]:
#Training and evaluating a simple 1D CNN on the Jena data
import keras

from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop
model = Sequential()
model.add(layers.Conv1D(32, 5, activation='relu',input_shape=(None, float_data.shape[-1])))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.compile(optimizer=RMSprop(), loss='mae')
history = model.fit_generator(train_gen,steps_per_epoch=500,epochs=20,validation_data=val_gen,validation_steps=val_steps)

Epoch 1/20
499/500 [============================>.] - ETA: 0s - loss: 0.4141

In [6]:
#Preparing higher-resolution data generators for the Jena dataset
step = 3
lookback = 720
delay = 144

train_gen = generator(float_data,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=200000,
                      shuffle=True,
                      step=step)
val_gen = generator(float_data,
                    lookback=lookback,
                    delay=delay,
                    min_index=200001,
                    max_index=300000,
                    step=step)
test_gen = generator(float_data,
                     lookback=lookback,
                     delay=delay,
                     min_index=300001,
                     max_index=None,
                     step=step)
val_steps = (300000 - 200001 - lookback) // 128
test_steps = (len(float_data) - 300001 - lookback) // 128


In [7]:
#Model combining a 1D convolutional base and a GRU layer
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop
model = Sequential()
model.add(layers.Conv1D(32, 5, activation='relu',
                        input_shape=(None, float_data.shape[-1])))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.GRU(32, dropout=0.1, recurrent_dropout=0.5))
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer=RMSprop(), loss='mae')
history = model.fit_generator(train_gen,
                                      steps_per_epoch=500,
                                      epochs=20,
                                      validation_data=val_gen,
                                      validation_steps=val_steps)


Using TensorFlow backend.
/Users/toni/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, None, 32)          2272      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 32)          5152      
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                6240      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 13,697
Trainable params: 13,697
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
500/500 [==============================] - 96s 193ms/step - loss: 0.3374 - val_loss: 0.2835
Epoch 2/20
500/500 [===============